In [1]:
import pygame as pg
import numpy as np
from numpy import random
import pdb


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Game should start with 9 tiles

these 9 cards always contain at least one 3 and at most 4 of either color. (4 ones,4, twos, 1 three)

The highest number you can draw is 3 ranks below your high card. If your high card is 24 or less, you only draw 3s. With a high card of 48, 6s start to show up. At 96, 12s come in and at 192, 24s appear.


The probability for each possible bonus card value seems to be equal. This means that as your high card increases it is more and more unlikely that you will see the highest possible bonus card. On the flip side, the value of bonus cards (even if they are announced in the coming update) becomes less and less predictable as the high card increases.


• The game uses an undrawn stack of 12 cards (four 1s, four 2s and four 3s). A stack is created at the beginning of a game and cards are drawn individually both to fill the board (see above) and to be added with every move. When a stack is depleted, a new one is created. I know I posited this theory before and then retracted it but it just fits to many beginnings of sequences (where I suspect the error-rate is lower) and I still believe in it. This has some consequences:


• I am not sure when a new high card takes effect. It either does so immediately (during a running stack) or at the start of the next stack (see below on the topic of stacks). Figuring this out is particularly nasty due to the problems listed in section 1 and the very low probability that you see an appropriate bonus card after the high card increases but before a new stack begins.


It could just as well be possible that bonus cards are completely unrelated to stacks. Instead, there is just a constant 5 % chance on every move that your next card is a bonus card

In [295]:
random.randint(0,4)

3

In [72]:
class Tiles:
    def __init__(self):
        
        self.possible_icons  = np.array([1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144, 12288])
        self.icons = dict()
        for index in self.possible_icons:
            self.icons[index] =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/_' + str(index) + '.png')
        self.stack = np.array([1,1,1,1,2,2,2,2,3])
        self.stack = self.stack[random.permutation(9)]    
            
        self.grid = np.zeros((16,),dtype = int)
        self.grid[0:9] = self.stack
        self.grid =  np.reshape(self.grid,(4,4))
        self.move = np.zeros((4,4),dtype = bool)
        
    def createStack(self):
        
        current_max = np.max(self.grid)
        
        max_position  = np.argwhere(self.possible_icons == current_max)
        bonus_cards = self.possible_icons[4:max_position[0][0]]
        
        self.stack = np.array([1,1,1,1,2,2,2,2,3])
        
        if bonus_cards.shape[0]>0:
            if random.rand(1)>.5:
                r =random.randint(0,len(bonus_cards))
                self.stack = np.append(self.stack,bonus_cards[r])
        rand_perm =  random.permutation(self.stack.shape[0])
        self.stack = self.stack[rand_perm]
        
        return self
        
    def drawStack(self):
        pos =0;
        for tile in self.stack:
            current_x = 400+ self.icons[tile].get_height()*np.floor(pos/4)
            current_y = self.icons[tile].get_height()*np.mod(pos,4)
            screen.blit(self.icons[tile],(current_x,current_y))
            
            pos +=1
                
            


    
    def determineMove(self,keys):
        current_move = [0, 0]
   
        if keys[pg.K_LEFT]:
            current_move = [0, -1]
  
        elif keys[pg.K_UP]:
            current_move = [-1, 0]
          
        elif keys[pg.K_RIGHT]:
            current_move = [0, 1]
            
        elif keys[pg.K_DOWN]:
            current_move = [1,0]
            
        return current_move
    def updatePosition(self,current_move):
        self.move = np.zeros((4,4),dtype = bool)
        x_range = np.array(range(4),dtype = int)
        y_range = np.array(range(4),dtype = int)
        if current_move[0] == 1:
            x_range = np.flip(x_range)
        if current_move[1] == 1:
            y_range = np.flip(y_range)
        
  
        for x in x_range:
            if x+current_move[0]>-1 and x+current_move[0]<4:
                for y in y_range:
                   
                    if y+current_move[1]>-1 and y+current_move[1]<4:
                        move_success = False
                        if self.grid[x,y] !=0:
                            new_pos = np.array([x,y]) + current_move
                            if self.grid[new_pos[0],new_pos[1]] == 0:
                                self.grid[new_pos[0],new_pos[1]]  = self.grid[x,y] 
                                self.grid[x,y] = 0
                                self.move[x,y] = True
                                
                            elif self.grid[new_pos[0],new_pos[1]] + self.grid[x,y] == 3:
                                self.grid[x,y] =0
                                self.grid[new_pos[0],new_pos[1]] =3
                                self.move[x,y] = True
                               
                            elif self.grid[new_pos[0],new_pos[1]] == self.grid[x,y]:
                                if self.grid[new_pos[0],new_pos[1]] + self.grid[x,y]>3:
                                    
                                    self.grid[new_pos[0],new_pos[1]]  = self.grid[x,y]*2
                                    self.grid[x,y]=0
                                    self.move[x,y] = True
                                    
                            

        return self
    
    def addTile(self,current_move):
        if np.sum(self.move,axis = None)>0:
      
            if current_move[0] ==-1:
                empty_spaces =  self.grid[3,:]==0
                moved_spaces =  np.sum(self.move,axis = 0)

            elif current_move[0] ==1:
                empty_spaces =  self.grid[0,:]==0
                moved_spaces =  np.sum(self.move,axis = 0)

            elif current_move[1] ==-1:
                empty_spaces =  self.grid[:,3]==0
                moved_spaces =  np.sum(self.move,axis = 1)

            elif current_move[1] ==1:
                empty_spaces =  self.grid[:,0]==0
                moved_spaces =  np.sum(self.move,axis = 1)

            new_spaces = empty_spaces * moved_spaces
            new_spaces = np.nonzero(new_spaces)
            new_spaces=new_spaces[0]

            if len(new_spaces)>0:
                r = np.random.permutation(len(new_spaces))
                new_spaces = new_spaces[r[0]]
                if current_move[0] == -1:
                    self.grid[3,new_spaces] = self.stack[0]
                elif current_move[0] == 1:
                    self.grid[0,new_spaces] = self.stack[0]
                elif current_move[1] == -1:
                    self.grid[new_spaces,3] = self.stack[0]
                elif current_move[1] == 1:
                    self.grid[new_spaces,0] = self.stack[0]
                  
                else:
                    print('No tile added!!!!!!!')
           
            else:
                pdb.set_trace()
                
            if self.stack.shape[0]>1:
                self.stack = np.delete(self.stack,0)
            else:
                print('created new stack')
                self =  self.createStack()
        else:
            print('no move possible... end of game!')
        return self
        
    def drawTile(self):
        for x in range(4):
            for y in range(4):
                if self.grid[x,y] !=0:
                    current_number = self.grid[x,y]
                    screen.blit(self.icons[current_number],(y*self.icons[current_number].get_width(),
                                      x*self.icons[current_number].get_height()))
        
        
   
    

In [39]:
game_tiles = Tiles()
game_tiles.move

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]])

In [73]:
running = True
current_x = 0
current_y = 0
screen = pg.display.set_mode((800,600))
screen.fill((0,0,255))
pg.display.set_caption("Threes!")


grid  =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/grid.png')
tile_list = [1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144]
start_num = 1

game_tiles = Tiles()

ex_state=True

while running:
    #pg.time.delay(100)
    screen.fill((0,0,255))
    screen.blit(grid,(0,0))
    keys = pg.key.get_pressed()
    if ex_state and sum(keys)>0:
        current_move = game_tiles.determineMove(keys)
        ex_state = False
    else:
        current_move = np.array([0,0])
    if sum(current_move) !=0:
        game_tiles = game_tiles.updatePosition(current_move)
        self = game_tiles.addTile(current_move)
    game_tiles.drawTile()
    game_tiles.drawStack()
        
    
            
    pg.display.update()

    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            pg.display.quit()
        if event.type == pg.KEYUP:
            ex_state = True;

    
      
            

created new stack
created new stack


In [66]:

pg.display.quit()



In [96]:
grid = np.arange(16)
grid =  np.reshape(grid,(4,4))
print(np.sum(grid,1))

[ 6 22 38 54]


In [43]:
a = False
if not(a):
    print('False')

False


In [25]:
a = np.array([0, 1, 2])
np.where(a < 4, a, -1)  # -1 is broadcast


array([0, 1, 2])

In [23]:
a.shape

(3, 3)